Set Up

In [1]:
from PW_explorer.load_worlds import load_worlds
from PW_explorer.run_clingo import run_clingo
from PW_explorer.helper import pw_slicer, rel_slicer
from PW_explorer.query import PWEQuery
from PW_explorer.dist_calc import PWEDistanceCalculation
from PW_explorer.visualize import PWEVisualization

In [2]:
import numpy as np
import pandas as pd
import random as random
import itertools 

In [3]:
%load_ext PWE_NB_Extension

Non-transitive dice is a kind of special dice that has the following Rules:

1.    The first player chooses a die from the set.
2.    The second player chooses one die from the remaining dice.
3.    Both players roll their die; the player who rolls the higher number wins.
4.    If non-transitive dice (and not fair dice), the probability of winning is biased in favor of second player
On the contrary, if the dice is transitive and unfair, first player should always be have more advantage.


We can see this phenomenon via the following example: there are three dice with 3 side: A (2,4,9), B(1,6,8), C(3,5,7).

In [4]:
#example for non-transitive dice 
dA = {1: 2, 2:4, 3:9}
dB = {1: 1, 2:6, 3:8}
dC = {1: 3, 2:5, 3:7}
dice = {"A":dA, "B":dB, "C":dC}


We can first analyze this instance via calculating the probability. 

### Analytical Approach

#### SQL

In [5]:
#SQLite -- see in non_transitive dice folder



#### Python Approach

In [6]:
def non_transitive_combo(dice):
    combo = list(itertools.combinations(dice, 2)) 
    combo_prob = {}
    for c in combo:
        d1 = dice[c[0]]
        d2 = dice[c[1]]
        combo_prob[c] = 0 
        for s1 in range(1, len(d1)+1):
            for s2 in range(1, len(d1)+1):
                if (d1[s1] > d2[s2]):
                    combo_prob[c] += 1
        combo_prob[c] /= len(d1)**2
        print
    return combo_prob

                    

In [7]:
non_transitive_combo(dice)

{('A', 'B'): 0.5555555555555556,
 ('A', 'C'): 0.4444444444444444,
 ('B', 'C'): 0.5555555555555556}

From the following output we see that there are three possible ways for the two players to choose:
If first player choose dice A, second player can choose dice C and has approximately 100% - 44.4% = 55.6% of chance beating dice A. 
If first player choose dice B, second player can choose dice A and has approximately 55.6% of chance beating B.
If first player choose dice C, second player can choose dice B and has approximately 55.6% of chance beating dice A.


#### Datalog

In [8]:
%%clingo --run -lci graph_instance --save_meta_data_to meta_data --saveto clingo_soln


% schema dice(diceID, side, value)

%dice(C, 1, 3). dice(C, 2, 5). dice(C, 3, 7). 
%dice(B, 1, 1). dice(B, 2, 6). dice(B, 3, 8). 
%dice(A, 1, 2). dice(A, 2, 4). dice(A, 3, 9).  % 3 dice instance
dice(3, 1, 3). dice(3, 2, 5). dice(3, 3, 7). 
dice(2, 1, 1). dice(2, 2, 6). dice(2, 3, 8). 
dice(1, 1, 2). dice(1, 2, 4). dice(1, 3, 9).    % 3 dice instance

% schema comp(Dice1, Dice2, Side1, Side2, Winner)

comp(D1,D2,S1,S2,D1) :- dice(D1,S1,R1), dice(D2,S2,R2), D1 < D2, R1>R2.
comp(D1,D2,S1,S2,D2) :- dice(D1,S1,R1), dice(D2,S2,R2), D1 < D2, R1<R2.

% schema subgame(Dice1, Dice2, NumWin)
subgame(D1, D2, N) :- N = 
    #count { D1, D2, S1, S2 :comp(D1, D2, S1, S2, D1)}, comp(D1, D2, S11, S22, R).

% :- subgame(D1, D2, N1), subgame(D2, D3, N2), subgame(D1, D3, N3), N1 < N2, N2 < N3.
% win_1(N) :- N = #count { D2, S1, S2 :comp(1, D2, S1, S2, 1)}.
% win_2(N) :- N = #count { D2, S1, S2 :comp(2, D2, S1, S2, 2)}.
% win_3(N) :- N = #count { D2, S1, S2 :comp(3, D2, S1, S2, 3)}.

Input:


'% schema dice(diceID, side, value)\n\n%dice(C, 1, 3). dice(C, 2, 5). dice(C, 3, 7).\n%dice(B, 1, 1). dice(B, 2, 6). dice(B, 3, 8).\n%dice(A, 1, 2). dice(A, 2, 4). dice(A, 3, 9).  % 3 dice instance\ndice(3, 1, 3). dice(3, 2, 5). dice(3, 3, 7).\ndice(2, 1, 1). dice(2, 2, 6). dice(2, 3, 8).\ndice(1, 1, 2). dice(1, 2, 4). dice(1, 3, 9).    % 3 dice instance\n\n% schema comp(Dice1, Dice2, Side1, Side2, Winner)\n\ncomp(D1,D2,S1,S2,D1) :- dice(D1,S1,R1), dice(D2,S2,R2), D1 < D2, R1>R2.\ncomp(D1,D2,S1,S2,D2) :- dice(D1,S1,R1), dice(D2,S2,R2), D1 < D2, R1<R2.\n\n% schema subgame(Dice1, Dice2, NumWin)\nsubgame(D1, D2, N) :- N =\n#count { D1, D2, S1, S2 :comp(D1, D2, S1, S2, D1)}, comp(D1, D2, S11, S22, R).\n\n% :- subgame(D1, D2, N1), subgame(D2, D3, N2), subgame(D1, D3, N3), N1 < N2, N2 < N3.\n% win_1(N) :- N = #count { D2, S1, S2 :comp(1, D2, S1, S2, 1)}.\n% win_2(N) :- N = #count { D2, S1, S2 :comp(2, D2, S1, S2, 2)}.\n% win_3(N) :- N = #count { D2, S1, S2 :comp(3, D2, S1, S2, 3)}.'

Output:


'Answer: 1\ndice(3,1,3) dice(3,2,5) dice(3,3,7) dice(2,1,1) dice(2,2,6) dice(2,3,8) dice(1,1,2) dice(1,2,4) dice(1,3,9) comp(2,3,2,1,2) comp(2,3,3,1,2) comp(1,3,2,1,1) comp(1,3,3,1,1) comp(2,3,2,2,2) comp(2,3,3,2,2) comp(1,3,3,2,1) comp(2,3,3,3,2) comp(1,3,3,3,1) comp(1,2,1,1,1) comp(1,2,2,1,1) comp(1,2,3,1,1) comp(1,2,3,2,1) comp(1,2,3,3,1) comp(2,3,1,1,3) comp(1,3,1,1,3) comp(2,3,1,2,3) comp(1,3,1,2,3) comp(1,3,2,2,3) comp(2,3,1,3,3) comp(2,3,2,3,3) comp(1,3,1,3,3) comp(1,3,2,3,3) comp(1,2,1,2,2) comp(1,2,2,2,2) comp(1,2,1,3,2) comp(1,2,2,3,2) subgame(2,3,5) subgame(1,3,4) subgame(1,2,5)\nSATISFIABLE\n\nModels       : 1\nCalls        : 1\nTime         : 0.002s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)\nCPU Time     : 0.002s'

In [9]:
pw_rels_dfs, rel_schemas, pw_objs = load_worlds(asp_output=clingo_soln, meta_data=meta_data, reasoner='clingo')

pw_rels_dfs.keys()


Number of Models: 1


dict_keys(['dice_3', 'comp_5', 'subgame_3'])

In [10]:
 pw_rels_dfs['subgame_3']

,pw,Dice1,Dice2,NumWin
0,1,2,3,5
1,1,1,3,4
2,1,1,2,5


From the ASP we know that the number of winning for each combination (with dice1 as the winner) is listed on the above dataframe. From it we know that P(dice 2 > dice1) > 0.5, P(dice 3 > dice2) > 0.5, and P(dice 1 > dice3) > 0.5, which shows that the set of dice are transitive dice. 


#### Simulation of Non-Transitive Dice

We can also use the simulation approach to mimic how two players perform in the game. For the following code, we simlate the game for N = 100000 times and see in each situation how would the dice output be look like. 


In [11]:


'''
@dice: list of die
@N   : number of trials 

'''  
def non_transitive_trials(dice, N):
    #combo = list(itertools.combinations(dice, 2)) 
    combo = [('A', 'B'), ('A', 'C'), ('B', 'C')]
    combo_res = {}
    #simulate all kinds of combination
    for choice in combo:
        d1 = dice[choice[0]]
        d2 = dice[choice[1]]
        sides = len(d1)
        
        #simulation of dice combo
        res = 0
        for i in range(N):
            v1 = d1[random.randint(1, sides)]
            v2 = d2[random.randint(1, sides)]
            if(v1 > v2):
                res += 1
        
        combo_res[choice] = res/N
    return combo_res
        
             

In [12]:
non_transitive_trials(dice, 100000)  

{('A', 'B'): 0.55774, ('A', 'C'): 0.44401, ('B', 'C'): 0.55493}

From the above result we see that in 100000 games, whichever dice the first player choose, the second player could always find a better dice that can have a higher winning chance overall. 